In [42]:
import pandas as pd
import copy
import warnings
import datetime as dt
import numpy as np
import plotly
import plotly.express as px

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [43]:
drivers_df = pd.read_csv("./f1db_csv/drivers.csv").drop(columns = "url")
races_df = pd.read_csv("./f1db_csv/races.csv")
constructors_df = pd.read_csv("./f1db_csv/constructors.csv")
races_df = pd.read_csv("./f1db_csv/races.csv")
laptimes_df = pd.read_csv("./f1db_csv/lap_times.csv")
results_df = pd.read_csv("./f1db_csv/results.csv")

# Clean some names and create new variables
# drivers_df
drivers_df["number"] = drivers_df["number"].replace({r"\N": None})
drivers_df["driverName"] = drivers_df["forename"].str.cat(drivers_df["surname"],sep = " ")
drivers_df = drivers_df.drop(columns = ["forename", "surname"])

# results_df
results_df["position"] = results_df["position"].replace({r"\N": None})

In [44]:
standings_df = pd.read_csv("./f1db_csv/driver_standings.csv")
driver_history_df = pd.read_csv("./f1db_csv/driver_history.csv")
colors_df = pd.read_csv("./f1db_csv/constructors_colors.csv")
colors_df.rename(columns={"name":"constructorName"},inplace=True)
colors_df

,Unnamed: 0,constructorId,constructorRef,constructorName,nationality,url,color
0,0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,#FF8700
1,1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber,#006EFF
2,2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,#0082FA
3,3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...,#FFF500
4,4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso,#469BFF
...,...,...,...,...,...,...,...
207,207,210,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team,#787878
208,208,211,racing_point,Racing Point,British,http://en.wikipedia.org/wiki/Racing_Point_F1_Team,#F596C8
209,209,212,alpha_tauri,Scuderia Alpha Tauri,Italian,http://en.wikipedia.org/wiki/Scuderia_Alpha_Tauri,#C8C8C8
210,210,213,alphatauri,AlphaTauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri,#C8C8C8


In [45]:
drivers_df

,driverId,driverRef,number,code,dob,nationality,driverName
0,1,hamilton,44,HAM,1985-01-07,British,Lewis Hamilton
1,2,heidfeld,None,HEI,1977-05-10,German,Nick Heidfeld
2,3,rosberg,6,ROS,1985-06-27,German,Nico Rosberg
3,4,alonso,14,ALO,1981-07-29,Spanish,Fernando Alonso
4,5,kovalainen,None,KOV,1981-10-19,Finnish,Heikki Kovalainen
...,...,...,...,...,...,...,...
848,850,pietro_fittipaldi,51,FIT,1996-06-25,Brazilian,Pietro Fittipaldi
849,851,aitken,89,AIT,1995-09-23,British,Jack Aitken
850,852,tsunoda,22,TSU,2000-05-11,Japanese,Yuki Tsunoda
851,853,mazepin,9,MAZ,1999-03-02,Russian,Nikita Mazepin


In [46]:
selectedSeason = 2021
races_season = races_df[races_df["year"]==2021]
raceid_list = races_season["raceId"]

#filter out lap times of the selected races
laptimes_df = laptimes_df[laptimes_df['raceId'].isin(raceid_list)]
driver_history_race = driver_history_df[driver_history_df['raceId'].isin(raceid_list)]
driver_history_race = pd.merge(driver_history_race,colors_df,on="constructorName")
driver_history_race

,Unnamed: 0_x,driverName,number,nationality_x,year,name,date,raceId,constructorName,position,minQualifyingTime,racePosition,fastestLapTime,wins,points,driverStanding,Unnamed: 0_y,constructorId,constructorRef,nationality_y,url,color
0,6892,Daniel Ricciardo,3.0,Australian,2021,Bahrain Grand Prix,2021-03-28,1052,McLaren,6,89.927,7.0,94.932,0,6.0,7,0,1,mclaren,British,http://en.wikipedia.org/wiki/McLaren,#FF8700
1,6893,Lando Norris,4.0,British,2021,Bahrain Grand Prix,2021-03-28,1052,McLaren,7,89.974,4.0,94.396,0,12.0,4,0,1,mclaren,British,http://en.wikipedia.org/wiki/McLaren,#FF8700
2,6912,Daniel Ricciardo,3.0,Australian,2021,Emilia Romagna Grand Prix,2021-04-18,1053,McLaren,6,74.826,6.0,79.341,0,14.0,7,0,1,mclaren,British,http://en.wikipedia.org/wiki/McLaren,#FF8700
3,6913,Lando Norris,4.0,British,2021,Emilia Romagna Grand Prix,2021-04-18,1053,McLaren,7,74.718,3.0,78.259,0,27.0,3,0,1,mclaren,British,http://en.wikipedia.org/wiki/McLaren,#FF8700
4,6932,Daniel Ricciardo,3.0,Australian,2021,Portuguese Grand Prix,2021-05-02,1054,McLaren,16,79.839,9.0,81.987,0,16.0,7,0,1,mclaren,British,http://en.wikipedia.org/wiki/McLaren,#FF8700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,7110,Yuki Tsunoda,22.0,Japanese,2021,Hungarian Grand Prix,2021-08-01,1062,AlphaTauri,16,77.919,6.0,78.394,0,18.0,13,210,213,alphatauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri,#C8C8C8
255,7129,Pierre Gasly,10.0,French,2021,Belgian Grand Prix,2021-08-29,1063,AlphaTauri,6,116.440,6.0,NaN,0,54.0,9,210,213,alphatauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri,#C8C8C8
256,7130,Yuki Tsunoda,22.0,Japanese,2021,Belgian Grand Prix,2021-08-29,1063,AlphaTauri,17,122.413,15.0,NaN,0,18.0,13,210,213,alphatauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri,#C8C8C8
257,7149,Pierre Gasly,10.0,French,2021,Dutch Grand Prix,2021-09-05,1064,AlphaTauri,4,69.478,4.0,74.818,0,66.0,8,210,213,alphatauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri,#C8C8C8


In [47]:
max_raceid = laptimes_df["raceId"].unique().max()
race_standings = standings_df[standings_df['raceId']==max_raceid]
race_standings.drop(columns=["driverStandingsId","raceId","points","wins","positionText"],inplace=True)
race_standings

,driverId,position
33185,1,2
33186,830,1
33187,822,3
33188,846,4
33189,815,5
33190,844,6
33191,817,9
33192,832,7
33193,852,13
33194,840,14


In [48]:

import plotly.graph_objects as go
drivers = pd.DataFrame(laptimes_df["driverId"].unique(),columns=["driverId"])
drivers = pd.merge(drivers,race_standings,on="driverId")
drivers.sort_values('position',inplace=True)

fig = go.Figure()
for driver in drivers["driverId"]:
    driver_entry = drivers_df[drivers_df["driverId"]==driver]
    driver_name = driver_entry['driverName']
    driver_name = driver_name.tolist()[0]
    driver_laptimes_season = laptimes_df[laptimes_df["driverId"]==driver]
    drivers_races_season = driver_history_race[driver_history_race["driverName"]==driver_name]
    max_raceid = drivers_races_season["raceId"].max()
    color = drivers_races_season[drivers_races_season["raceId"]==max_raceid]["color"]
    color = color.tolist()[0]
    driver_laptimes_season.sort_values('position',inplace=True)
    violin = go.Violin(y=driver_laptimes_season["position"],bandwidth=0.5,points=False,meanline_visible=True,opacity=0.8,hoveron="violins",x0=driver_name,name=driver_name,line_color='black',fillcolor=color)
    fig.add_trace(violin)
fig.update_yaxes(type='category')
fig.update_yaxes(categoryorder='array', categoryarray= list(range(1,laptimes_df['position'].max(),1)))
fig.update_traces(spanmode="hard",line_width=1,width=1)
fig.update_layout(
    yaxis= dict(tickmode='linear',tick0=1,dtick=1)
)
fig.update_layout(plot_bgcolor="#323130",
			paper_bgcolor="#323130",font=dict(color="white"),
			yaxis_title="Position",
            title='Driver position for each lap',
			margin = dict(l=20,r=20,t=20,b=20)
			)
fig.show()